In [10]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split

In [11]:
# Load the dataset and train, val, test splits
print("Loading datasets...")
my_transformer = transforms.Compose([
    transforms.ToTensor(),
])
MNIST_train = datasets.MNIST('.', download=True, train=True, transform=my_transformer)
MNIST_test = datasets.MNIST('.', download=True, train=False, transform=my_transformer)


Loading datasets...


In [12]:
BATCH_SIZE = 100
trainloader = DataLoader(MNIST_train, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(MNIST_test, batch_size=BATCH_SIZE)

In [13]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5),                                            
            nn.MaxPool2d(kernel_size=2),   
            nn.ReLU(),         
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),                        
            nn.MaxPool2d(2),   
            nn.ReLU(),                
        )
        self.out = nn.Linear(1152, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
model = Network().to(device)
criterion = nn.CrossEntropyLoss() # Specify the loss layer
optimizer = optim.AdamW(model.parameters(), lr=0.001)
num_epoch = 5 

In [15]:
def train(model, train_loader, num_epoch = 5): # Train the model
    training_loss = []
    #validation_loss = []
    print("Start training...")
    model.train() # Set the model to training mode
    for i in range(num_epoch):
        training_running_loss = []
        for batch, label in tqdm(train_loader):
            batch = batch.to(device)
            label = label.to(device)
            optimizer.zero_grad() # Clear gradients from the previous iteration
            pred = model(batch) # This will call Network.forward() that you implement
            loss = criterion(pred, label) # Calculate the loss
            training_running_loss.append(loss.item())
            loss.backward() # Backprop gradients to all tensors in the network
            optimizer.step() # Update trainable weights
        print("Epoch {} loss:{}".format(i+1,np.mean(training_running_loss))) # Print the average loss for this epoch
        training_loss.append(np.mean(training_running_loss))
        
    print("Done!")
    return training_loss




In [16]:
def evaluate(model, loader): # Evaluate accuracy on validation / test set
    running_loss = []
    model.eval() # Set the model to evaluation mode
    correct = 0
    with torch.no_grad(): # Do not calculate grident to speed up computation
        for batch, label in tqdm(loader):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            loss = criterion(pred, label) # Calculate the loss
            running_loss.append(loss.item())
            correct += (torch.argmax(pred,dim=1)==label).sum().item()
    acc = correct/len(loader.dataset)
    print("Evaluation accuracy: {}".format(acc))
    return acc, np.mean(running_loss)

#run training and validation for training. 
training_loss = train(model, trainloader, 5)

#save the model
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/bestModelUpdated.pt")

print("Evaluate on test set")
evaluate(model, testloader)

Start training...


100%|██████████| 600/600 [00:06<00:00, 86.22it/s]


Epoch 1 loss:0.24570397583457332


100%|██████████| 600/600 [00:07<00:00, 76.75it/s]


Epoch 2 loss:0.06371192975745847


...

100%|██████████| 600/600 [00:07<00:00, 81.48it/s]


Epoch 5 loss:0.03150026157265529
Done!
Evaluate on test set


100%|██████████| 100/100 [00:01<00:00, 70.71it/s]

Evaluation accuracy: 0.9901


(0.9901, 0.030249218420358374)